In [1]:
import praw

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

CLIENT_ID = os.getenv("R_CLIENT_ID")
CLIENT_SECRET = os.getenv("R_CLIENT_SECRET")
USER_AGENT = os.getenv("R_USER_AGENT")
USERNAME = os.getenv("R_USERNAME")
PASSWORD = os.getenv("R_PASSWORD")

In [3]:
reddit_instance = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT,
    username=USERNAME,
    password=PASSWORD
)

In [4]:
subreddit = reddit_instance.subreddit('portugal2')
subreddit

Subreddit(display_name='portugal2')

In [5]:
import pandas as pd
from datetime import datetime

top_posts = subreddit.top(limit=1000, time_filter="year")

posts_df = []
for post in top_posts:
    # Converte o timestamp Unix para um formato de data legível
    post_date = datetime.utcfromtimestamp(post.created_utc).strftime('%d-%m-%Y')

    # Adiciona os dados do post à lista
    posts_df.append({
        'Title': post.title,
        'Content': post.selftext,
        'Number of Comments': post.num_comments,
        'Created': post_date  # Data de criação
    })
posts_df = pd.DataFrame(posts_df)
posts_df.to_excel('reddit.xlsx', index=False)

In [7]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Carregar as stopwords em português
stop_words = set(stopwords.words('portuguese'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Converter para minúsculas
    text = text.lower()
    
    # Remover caracteres especiais, mantendo apenas letras e números
    text = re.sub(r'[^a-zá-úãõàâäçéêíóôúùñ\s0-9]', '', text)
    
    # Remover números 
    text = re.sub(r'\d+', '', text)
    
    # Tokenizar as palavras
    words = word_tokenize(text)
    
    # Remover stopwords
    words_filtered = [word for word in words if word not in stop_words]
    
    # Lematizar as palavras
    #words_lemmatized = [lemmatizer.lemmatize(word) for word in words_filtered]
    
    # Juntar as palavras lematizadas de volta em uma string
    #return ' '.join(words_lemmatized)

# Aplicar o pré-processamento ao conteúdo
posts_df['Content'] = posts_df['Content'].apply(lambda x: clean_text(str(x)))

# Salvar o DataFrame processado em um arquivo Excel
posts_df.to_excel('reddit_cleaneds.xlsx', index=False)

# Exibir as primeiras linhas do DataFrame processado
print(posts_df.head())

                                    Title  \
0  Namorado da Cristina Ferreira zangado.   
1                  Vamos limpar Portugal!   
2      Táxi falso no aeroporto de Lisboa    
3                             É +/- isto    
4                  Golden explanation...    

                                             Content  Number of Comments  \
0                                                                     93   
1  nata pessoas vão tornar possivel cristina rodr...                 610   
2                        un turistas serem enganados                 137   
3                                                                    132   
4                                                                     67   

      Created                                    Cleaned_Content  
0  21-05-2024                                                     
1  14-03-2024  natar pessoa ir tornar possivel cristina rodri...  
2  10-07-2024                             um turista ser enganar  
3  3

In [6]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('portuguese'))
# Função para remover stopwords, números e deixar tudo em minúsculas
def clean_text(text):
    # Converter o texto para minúsculas
    text = text.lower()
    
    # Remover números
    text = re.sub(r'\d+', '', text)
    
    # Tokenizar o texto em palavras
    words = word_tokenize(text)
    
    # Remover stopwords
    words_filtered = [word for word in words if word not in stop_words]
    
    # Juntar as palavras filtradas em uma string
    return ' '.join(words_filtered)

# Aplicar a função ao dataframe
posts_df['Content'] = posts_df['Content'].apply(lambda x: clean_text(str(x)))

# Salvar o DataFrame processado em um arquivo Excel
posts_df.to_excel('reddit_cleaned.xlsx', index=False)